# simulating the data

## 1. Def a function to generate binary data files with different 0 ratio

In [1]:
def gen_random_bi(zeropercentage):
    import numpy as np
    random_bi = np.random.choice([0,1], size = 8 * 1000 *1000 * 100, replace = True, p = [zeropercentage, 1 - zeropercentage])
    random_bi = np.packbits(random_bi)
    file_name = "zero_"+ str(zeropercentage)
    with open(file_name, "wb") as BI:
        BI.write(random_bi)
        BI.close()
        
for i in [0.5,0.6,0.7,0.8,0.9,1.0]:
    print(i)
    gen_random_bi(i)
    

0.5
0.6
0.7
0.8
0.9
1.0


## 2. Generate DNA sequence

In [2]:
import numpy as np

Nt = ['A', 'G', 'C', 'T']
random_DNA = np.random.choice(Nt, size = 100000000, replace = True, p = [0.25, 0.25, 0.25, 0.25])
random_DNA.tostring()

with open("nt_seq.fa", "w") as NT:
    NT.write("".join(random_DNA))
    NT.close()



In [3]:
import numpy as np
Aa = ['G','A','V','L','I','P','F','Y','W','S','T','C','M','N','Q','D','E','K','R','H']
pr = []
for i in range(0, 20):
    pr.append(0.05)
random_Pro = np.random.choice(Aa, size = 100000000, replace = True, p = pr )
random_Pro.tostring()
with open("protein_seq.fa", "w") as PR:
    PR.write("".join(random_Pro))
    PR.close()


# Compressing the data

### gzip = DEFLATE = Huffman + Lempel-Ziv

### bzip = Burrows-Wheeler

In [22]:
def comp_file(filename):
    import subprocess 
    out_file = filename + '.gz'
    time_dic = {}
    pro1 = subprocess.run("time gzip -c %s > %s"%(filename,out_file),
                           stderr = subprocess.PIPE, 
                           shell = True)
    time_dic["gz"] = pro1.stderr.decode()
    
    out_file = filename + '_bzip2' + '.bz2'
    pro2 = subprocess.run("time bzip2 -c -k %s > %s"%(filename,out_file),
                           stderr = subprocess.PIPE, 
                           shell = True)                     
    time_dic["bzip2"] = pro2.stderr.decode()
    
    out_file = filename + '_pbzip2' + '.bz2'
    pro3 = subprocess.run("time pbzip2 -c -k %s > %s"%(filename,out_file),
                           stderr = subprocess.PIPE, 
                           shell = True) 
    time_dic["pbzip2"] = pro3.stderr.decode()
    print(time_dic)
    return time_dic

## 2. write a for loop to compress binary files.

In [23]:
times = []
for i in ['zero_0.5','zero_0.6','zero_0.7','zero_0.8','zero_0.9','zero_1.0']:
    time = comp_file(i)
    times.append(time)
    

{'gz': '\nreal\t0m3.512s\nuser\t0m3.405s\nsys\t0m0.104s\n', 'bzip2': '\nreal\t0m13.216s\nuser\t0m13.116s\nsys\t0m0.096s\n', 'pbzip2': '\nreal\t0m2.109s\nuser\t0m26.010s\nsys\t0m0.704s\n'}
{'gz': '\nreal\t0m4.334s\nuser\t0m4.229s\nsys\t0m0.104s\n', 'bzip2': '\nreal\t0m12.403s\nuser\t0m12.296s\nsys\t0m0.104s\n', 'pbzip2': '\nreal\t0m1.966s\nuser\t0m24.121s\nsys\t0m0.557s\n'}
{'gz': '\nreal\t0m7.125s\nuser\t0m6.934s\nsys\t0m0.072s\n', 'bzip2': '\nreal\t0m11.185s\nuser\t0m11.038s\nsys\t0m0.076s\n', 'pbzip2': '\nreal\t0m1.612s\nuser\t0m20.328s\nsys\t0m0.580s\n'}
{'gz': '\nreal\t0m17.061s\nuser\t0m16.925s\nsys\t0m0.132s\n', 'bzip2': '\nreal\t0m10.312s\nuser\t0m10.206s\nsys\t0m0.104s\n', 'pbzip2': '\nreal\t0m1.425s\nuser\t0m17.232s\nsys\t0m0.554s\n'}
{'gz': '\nreal\t0m26.633s\nuser\t0m26.372s\nsys\t0m0.068s\n', 'bzip2': '\nreal\t0m9.828s\nuser\t0m9.775s\nsys\t0m0.052s\n', 'pbzip2': '\nreal\t0m1.201s\nuser\t0m14.960s\nsys\t0m0.490s\n'}
{'gz': '\nreal\t0m0.641s\nuser\t0m0.575s\nsys\t0m0.028s\n'

## deal with the output of time command, add real time and user time together

Use regular expression

In [25]:
import re

def find_time(time_str):
    ret = re.findall(r"m(.*)s",time_str)
    return ret[0]

for i in times:
    time_list = i["gz"].split()
    i["gz"] = float(find_time(time_list[1])) + float(find_time(time_list[3]))
    time_list = i["bzip2"].split()
    i["bzip2"] = float(find_time(time_list[1])) + float(find_time(time_list[3]))
    time_list = i["pbzip2"].split()
    i["pbzip2"] = float(find_time(time_list[1])) + float(find_time(time_list[3]))

In [26]:
import pandas as pd

pd.DataFrame(times, index = ["zero0.5", "zero0.6", "zero0.7", "zero0.8", "zero0.9", "zero1.0"])

,bzip2,gz,pbzip2
zero0.5,26.332,6.917,28.119
zero0.6,24.699,8.563,26.087
zero0.7,22.223,14.059,21.940
zero0.8,20.518,33.986,18.657
zero0.9,19.603,53.005,16.161
zero1.0,1.745,1.216,1.922


### table of compression efficience ( the compression efficience information about random DNA and protein will be in another table in the "compress real data" section )

In [42]:
data = {
    'gz':['100MB','99.7MB','89.3MB',"77.4MB","56MB",'91.7kB'],
    'bzip':['100MB','100MB','95.1MB','82.6MB','58.3MB','113B'],
    'pbzip':['100MB',"100MB",'95.1MB','82.6MB','58.3MB','5.38kB']
}
pd.DataFrame(data, index = ['zero 0.5', 'zero 0,6', 'zero 0.7', 'zero 0.8', 'zero 0.9', 'zero 1.0'])


,gz,bzip,pbzip
zero 0.5,100MB,100MB,100MB
"zero 0,6",99.7MB,100MB,100MB
zero 0.7,89.3MB,95.1MB,95.1MB
zero 0.8,77.4MB,82.6MB,82.6MB
zero 0.9,56MB,58.3MB,58.3MB
zero 1.0,91.7kB,113B,5.38kB


In [27]:
time_nt = comp_file("nt_seq.fa")

time_list = time_nt["gz"].split()
time_nt["gz"] = float(find_time(time_list[1])) + float(find_time(time_list[3]))
time_list = time_nt["bzip2"].split()
time_nt["bzip2"] = float(find_time(time_list[1])) + float(find_time(time_list[3]))
time_list = time_nt["pbzip2"].split()
time_nt["pbzip2"] = float(find_time(time_list[1])) + float(find_time(time_list[3]))

{'gz': '\nreal\t0m23.604s\nuser\t0m23.571s\nsys\t0m0.032s\n', 'bzip2': '\nreal\t0m9.550s\nuser\t0m9.506s\nsys\t0m0.044s\n', 'pbzip2': '\nreal\t0m1.099s\nuser\t0m14.268s\nsys\t0m0.512s\n'}


In [28]:
pd.DataFrame(time_nt, index = ["nucleotides"])

,gz,bzip2,pbzip2
nucleotides,47.175,19.056,15.367


In [29]:
time_pr = comp_file("protein_seq.fa")
time_list = time_pr["gz"].split()
time_pr["gz"] = float(find_time(time_list[1])) + float(find_time(time_list[3]))
time_list = time_pr["bzip2"].split()
time_pr["bzip2"] = float(find_time(time_list[1])) + float(find_time(time_list[3]))
time_list = time_pr["pbzip2"].split()
time_pr["pbzip2"] = float(find_time(time_list[1])) + float(find_time(time_list[3]))

{'gz': '\nreal\t0m4.371s\nuser\t0m4.298s\nsys\t0m0.072s\n', 'bzip2': '\nreal\t0m9.642s\nuser\t0m9.553s\nsys\t0m0.088s\n', 'pbzip2': '\nreal\t0m1.171s\nuser\t0m14.683s\nsys\t0m0.568s\n'}


In [30]:
pd.DataFrame(time_pr, index = ["protein"])

,gz,bzip2,pbzip2
protein,8.669,19.195,15.854


# Questions

### 1. Which algorithm achieves the best level of compression on each file type?

    For nucleotide fasta file and protein fasta file, the bzip2 and pbzip2 got the same size compressed file. So they both achieve the best level of compression on these two kinds of files.
    
    For binary files, when the zero rate is not 100%, gzip's perforance was better then bzip2 and pbzip2. But when the zero rate is 100%, bzip2 achieves the best level of compression.
    
   
### 2. Which algorithm is the fastest?
    
    For nucleotide fasta file, the fast one is bzip2;
    
    For protein fasta file, the fast one is gz;
    
    For binary files, when 0 is 80% and 90%, pbzip2 is the fast;
    
    When 0 is 50%, %60, 70%, 100%  gz is the fast.
    
    
    

### 3. What is the difference between bzip2 and pbzip2? Do you expect one to be faster and why?
    
    pbzip2 is a parallel implementation of the bzip2 block-sorting file compressor that uses pthreads and achieves near-linear speedup on SMP machines. 
    
    So if I call multiple cores to execute pbzip, I think it will be faster than bzip, but I didn't do so in the previous test.


### 4. How does the level of compression change as the percentage of zeros increases? Why does this happen?

    When the proportion of zeros gradually increases, the level of compression becomes better. Because we know a sequence with uniformly distribution has the largest Shannon entropy.
    
    

### 5. What is the minimum number of bits required to store a single DNA base?

    log2(4) = 2

### 6. What is the minimum number of bits requied to store an amina acid letter?

    log2(20) = 4.32
    Because it must be an integer
    

### 7. In your tests, how many bits did gzip and bzip2 actually require to store your random DNA and protein sequences?

    bzip: 1. DNA.bz2 : 27.3MB
          2. Protein.bz2 : 55.3MB
      
    gzip: 1. DNA.gz :29.2MB
          2. Protein.gz : 60.6MB
      
### 8. Are gzip and bzip2 performing well on DNA and proteins?
    
    I think neither of them do very well on DNA, Because if the random DNA is encoded in an ideal code (Use exactly 2 bits to store every base), the file size should be 25Mb.
    
    bzip is performing well on proteins. Because 55.3 Mb is very close to 54Mb(the size of file encoded in ideal code).
    gzip performs worse than bzip.
    
    
    





    

# Compressing the real data

## 1. Querying biological databases and download 10 sequences of gp120 (Envelope glycoprotein GP120) from 10 HIV isolates.

I got a single multi-FASTA file from NCBI. I named it as "gp120_HIV_10_isolates.fasta" and uploaded to my dictionary on the server.

## 2. Compress the multi-FASTA using gzip and bzip2

## Question：In priori, do you expect to abchieve better or worse compression here than random data? Why?

    Yes, I expect to a better compression here. Because the 10 sequence in this fasta file are extremly similar to each other, they are homologous. So it can be compressed to a smaller size.

In [31]:
time_realdata = comp_file("gp120_HIV_10_isolates.fasta")

{'gz': '\nreal\t0m0.001s\nuser\t0m0.001s\nsys\t0m0.000s\n', 'bzip2': '\nreal\t0m0.003s\nuser\t0m0.002s\nsys\t0m0.000s\n', 'pbzip2': '\nreal\t0m0.015s\nuser\t0m0.003s\nsys\t0m0.000s\n'}


In [32]:
time_list = time_realdata["gz"].split()
time_realdata["gz"] = float(find_time(time_list[1])) + float(find_time(time_list[3]))
time_list = time_realdata["bzip2"].split()
time_realdata["bzip2"] = float(find_time(time_list[1])) + float(find_time(time_list[3]))
time_list = time_realdata["pbzip2"].split()
time_realdata["pbzip2"] = float(find_time(time_list[1])) + float(find_time(time_list[3]))

In [35]:
pd.DataFrame(time_realdata, index = ["realdata time"])

,gz,bzip2,pbzip2
realdata time,0.002,0.005,0.018


## a table of file size for compression files of  realdata and random sequences files.

In [41]:
filesize = {
    'random protein': ["100MB", "60.6MB","60.6%","55.3MB","55.3%","55.3MB",'55.3%'],
    'random DNA': ['100MB','29.2MB','29.2%', '27.3MB','27.3%','27.3MB','27.3%'],
    'real data': ['5.63KB','893B','15.86%','1.05KB','18.65%','1.05Kb','18.65%']
}
pd.DataFrame(filesize, index = ["Original file", "gzip","gzip ratio", "bzip",'bzip ratio',"pbzip",'pbzip ratio'])

,random protein,random DNA,real data
Original file,100MB,100MB,5.63KB
gzip,60.6MB,29.2MB,893B
gzip ratio,60.6%,29.2%,15.86%
bzip,55.3MB,27.3MB,1.05KB
bzip ratio,55.3%,27.3%,18.65%
pbzip,55.3MB,27.3MB,1.05Kb
pbzip ratio,55.3%,27.3%,18.65%


### Question : How does the compression ratio of this file compare to random data?

    The compression files of real data is smaller

# Estimating compression of 1000 terabytes

#### Let’s make some assumptions about the contents of the data at your biotech company. Most of the data, say 80%, is re-sequencing of genomes and plasmids that are very similar to each other. Another 10% might be protein sequences, and the last 10% are binary microscope images which we’ll assume follow the worst-case scenario of being completely random.Given the benchmarking data you obtained in this lab, which algorithm do you propose to use for each type of data? Provide an estimate for the fraction of space you can save using your compression scheme. How much of a bonus do you anticipate receiving this year?

1. For those 80% re-sequencing of genomes and plasmids, I want to use gzip, and because these datas are similar to each other, so I assume that the compression ratio will be similar to the compression ratio of the real data in the previous test. It will save 84.14% space. 

2. For the 10% protein sequences, I want to use bzip2. According to the compression ratio of the random protein sequence in the previous test, I assume it will save 44.7% space.

3. For the 10% binary sequence, we will follow the worst-case scenario, it will be completely random. And in the worst condition, the 1 and 0 will be 1:1, then the compression can't save space.

    80% * 84.14% = 67.31%
    10% * 44.7%  = 4.47%
    67.31% + 4.47% = 71.78%
    
    0.7178 *1000*50 = 35890$
    
